# Zgodovina šifre

### Cezarjeva šifra

In [ ]:
def caesar_decrypt(ciphertext, shift):
    result = ""
    for ch in ciphertext:
        if ch.isalpha():
            # prestavi v 0–25
            base = ord('A') if ch.isupper() else ord('a')
            result += chr((ord(ch) - base - shift) % 26 + base)
        else:
            result += ch
    return result

cipher = input("Vnesi šifrirano besedilo: ")

for s in range(26):
    print(f"Shift {s}: {caesar_decrypt(cipher, s)}")

### Zamenjalna šifra + Frekvenčna analiza

In [ ]:
from collections import Counter

# standardna frekvenca črk v angleščini
english_frequency_order = "ETAOINSHRDLCUMWFGYPBVKJXQZ"

cipher = input("Vnesi šifrirano besedilo (samo črke): ").upper()

# 1. Frekvenčna analiza
counts = Counter(cipher)
sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)

print("\nPogostost črk:")
for letter, cnt in sorted_counts:
    print(letter, ":", cnt)

# 2. Predlagana preslikava (najpogostejša -> E, nato T, A ...)
print("\nPredlagana preslikava:")
mapping = {}
for i, (cipher_letter, _) in enumerate(sorted_counts):
    if i < len(english_frequency_order):
        mapping[cipher_letter] = english_frequency_order[i]

for c, p in mapping.items():
    print(f"{c} → {p}")

# 3. Delno dešifriranje
plaintext_guess = ""
for ch in cipher:
    plaintext_guess += mapping.get(ch, "?")  # neznane črke označimo z '?'

print("\nPredlagano dešifriranje:")
print(plaintext_guess)


### Vigenère + Gnetljivost tajnopisa

In [ ]:
# Funkcija za izračun IC:
from collections import Counter

def index_of_coincidence(text):
    counts = Counter(text)
    N = len(text)
    if N <= 1:
        return 0

    ic = sum(f * (f - 1) for f in counts.values()) / (N * (N - 1))
    return ic


#Glavni program: iskanje dolžine ključa
cipher = input("Vnesi Vigenèro šifrirano besedilo: ").replace(" ", "").upper()

max_key_len = 12  # lahko spremeniš
ics = []

print("\nIC za poskusne dolžine ključev:")

for key_len in range(1, max_key_len + 1):
    ic_values = []
    # deli besedilo v kolone (po modulu)
    for offset in range(key_len):
        column = cipher[offset::key_len]
        ic_values.append(index_of_coincidence(column))

    avg_ic = sum(ic_values) / len(ic_values)
    ics.append((key_len, avg_ic))
    print(f"K = {key_len} → IC = {avg_ic:.4f}")

# izbira najverjetnejše dolžine ključa (IC najbližji ~0.065)
best = max(ics, key=lambda x: x[1])

print(f"\nNajverjetnejša dolžina ključa: {best[0]}")


### Avtomatski napad na Vigenère (ključ + dešifriranje)
1) Index of Coincidence (IC) za določitev dolžine ključa
2) Frekvenčna analiza za razvozlavanje posameznih črk ključa

In [3]:
from collections import Counter
import string

# -------------------------------
#   Index of Coincidence
# -------------------------------
def index_of_coincidence(text):
    counts = Counter(text)
    N = len(text)
    if N <= 1:
        return 0
    ic = sum(f * (f - 1) for f in counts.values()) / (N * (N - 1))
    return ic

# -------------------------------
#   Caesar dešifriranje za eno kolono
# -------------------------------
def caesar_shift(text, shift):
    result = ""
    for ch in text:
        if ch.isalpha():
            base = ord('A')
            result += chr((ord(ch) - base - shift) % 26 + base)
        else:
            result += ch
    return result

# -------------------------------
#   Frekvenčna analiza za eno kolono
#   (najbolj pogosta črka naj bo 'E')
# -------------------------------
def guess_caesar_shift(column):
    freq = Counter(column)
    most_common_char, _ = freq.most_common(1)[0]
    shift = (ord(most_common_char) - ord('E')) % 26
    return shift

# -------------------------------
#   Avtomatsko ugibanje dolžine ključa
# -------------------------------
def guess_key_length(cipher, max_len=12):
    ics = []
    for key_len in range(1, max_len + 1):
        ic_values = []
        for offset in range(key_len):
            column = cipher[offset::key_len]
            ic_values.append(index_of_coincidence(column))

        avg_ic = sum(ic_values) / len(ic_values)
        ics.append((key_len, avg_ic))

    # V angleščini je IC za navadno besedilo ~0.065
    best = max(ics, key=lambda x: x[1])
    return best[0], ics

# -------------------------------
#   Pridobivanje celotnega ključa
# -------------------------------
def recover_key(cipher, key_length):
    key = ""
    for i in range(key_length):
        column = cipher[i::key_length]
        shift = guess_caesar_shift(column)
        key += chr(ord('A') + shift)
    return key

# -------------------------------
#   Glavno dešifriranje
# -------------------------------
def vigenere_decrypt(cipher, key):
    plaintext = []
    key = key.upper()
    key_len = len(key)
    j = 0

    for ch in cipher:
        if ch.isalpha():
            shift = ord(key[j % key_len]) - ord('A')
            base = ord('A')
            plaintext.append(chr((ord(ch) - base - shift) % 26 + base))
            j += 1
        else:
            plaintext.append(ch)

    return "".join(plaintext)

# -------------------------------
#   MAIN
# -------------------------------
cipher = input("Vnesi Vigenère šifrirano besedilo: ").replace(" ", "").upper()

# 1) Ugotovi dolžino ključa
best_len, ic_values = guess_key_length(cipher)

print("\nIC vrednosti za možne dolžine:")
for k, ic in ic_values:
    print(f"K={k}: IC={ic:.4f}")

print(f"\nNajverjetnejša dolžina ključa: {best_len}")

# 2) Ugotovi ključ
key = recover_key(cipher, best_len)
print(f"\nUgotovljeni ključ: {key}")

# 3) Dešifriraj
plaintext = vigenere_decrypt(cipher, key)
print("\nDešifrirano besedilo:")
print(plaintext)



IC vrednosti za možne dolžine:
K=1: IC=0.0303
K=2: IC=0.0000
K=3: IC=0.0556
K=4: IC=0.0000
K=5: IC=0.2000
K=6: IC=0.0000
K=7: IC=0.0000
K=8: IC=0.0000
K=9: IC=0.0000
K=10: IC=0.0000
K=11: IC=0.0000
K=12: IC=0.0000

Najverjetnejša dolžina ključa: 5

Ugotovljeni ključ: HTBKL

Dešifrirano besedilo:
EEEEEOLEHCAY


## Enkratna podloga

In [ ]:
#Program avtomatsko izvede napad na One-Time Pad, ki je bil ponovno uporabljen, tako da:
    # iz XOR obeh šifratov dobiš P1 XOR P2,
    # ugibaš del P1,
    # iz tega izračunaš podlogo K,
    # in potem dešifriraš ustrezen del P2.
    
def hex_to_bytes(hex_string):
    return bytes.fromhex(hex_string)

def xor_bytes(b1, b2):
    return bytes(a ^ b for a, b in zip(b1, b2))

# Podani šifrati
C1_hex = "3a1f756b2a097d14"
C2_hex = "5772110b476c1079"

C1 = hex_to_bytes(C1_hex)
C2 = hex_to_bytes(C2_hex)

# 1) C1 XOR C2 (P1 XOR P2)
xor_result = xor_bytes(C1, C2)
print("C1 XOR C2 =", xor_result.hex())

# 2) predpostavka P1 = "HELLO"
P1_guess = "HELLO".encode("ascii")
K = xor_bytes(C1[:len(P1_guess)], P1_guess)
print("K (prvih 5 bajtov) =", K.hex())

# 3) izračun P2
P2 = xor_bytes(C2[:len(P1_guess)], K)
print("P2 (prvih 5 bajtov) =", P2.hex(), "→", P2.decode(errors="replace"))


C1 XOR C2 = 6d6d64606d656d6d
K (prvih 5 bajtov) = 725a392765
P2 (prvih 5 bajtov) = 2528282c22 → %((,"


## PRG in LCG
Implementiraj Linearni Kongruenčni Generator (LCG) z parametri m, a, c. Uporabi začetni seed. Generiraj N bajtov (ali bitov) keystreama tako, da vsak LCG izhod spremeniš v en byte (npr. state % 256).
Napiši funkcijo, ki vrne keystream dolžine N in funkcijo, ki izkešira (XOR) s plaintextom. Preizkusi s kratkim plaintextom in prikazuj entropijo/pogostost bajtov.

In [ ]:
# prg_lcg.py
import math
from collections import Counter

def lcg_bytes(seed, a, c, m, n_bytes):
    state = seed % m
    out = bytearray()
    for _ in range(n_bytes):
        state = (a * state + c) % m
        out.append(state % 256)
    return bytes(out)

def xor_bytes(b1: bytes, b2: bytes) -> bytes:
    return bytes(x ^ y for x, y in zip(b1, b2))

def byte_entropy(data: bytes) -> float:
    # approx Shannon entropy per byte
    cnt = Counter(data)
    N = len(data)
    ent = 0.0
    for v in cnt.values():
        p = v / N
        ent -= p * math.log2(p)
    return ent

if __name__ == "__main__":
    # primer parametrov (šibki za kriptografijo - za učenje)
    m = 2**31
    a = 1103515245
    c = 12345
    seed = 42

    ks = lcg_bytes(seed, a, c, m, 64)
    print("Keystream (hex):", ks.hex())
    print("Entropija (bitov na byte):", byte_entropy(ks))

    plaintext = b"THIS IS A SECRET MESSAGE"
    ciphertext = xor_bytes(plaintext, ks[:len(plaintext)])
    recovered = xor_bytes(ciphertext, ks[:len(plaintext)])
    print("Plain:", plaintext)
    print("Cipher (hex):", ciphertext.hex())
    print("Recovered:", recovered)


: 

## PRG iz HMAC-SHA256 (KDF-style)
Naredi varnejši PRG tako, da iz vira seed (nek byte string) generiraš proizvedeni keystream z iterativnim HMAC-SHA256 (podoben HKDF-expand). Implementiraj funkcijo prg_hmac(seed, n_bytes) in jo uporabi za XOR šifriranje.

In [ ]:
# prg_hmac.py
import hmac, hashlib

def prg_hmac(seed: bytes, n_bytes: int) -> bytes:
    """Generiraj n_bytes uporabi hmac-sha256 kot PRG (HKDF-like expand)."""
    out = bytearray()
    counter = 1
    last = b""
    while len(out) < n_bytes:
        mac = hmac.new(seed, last + bytes([counter]), hashlib.sha256).digest()
        out.extend(mac)
        last = mac
        counter += 1
    return bytes(out[:n_bytes])

def xor_bytes(b1: bytes, b2: bytes) -> bytes:
    return bytes(x ^ y for x, y in zip(b1, b2))

if __name__ == "__main__":
    seed = b"shared secret seed"
    ks = prg_hmac(seed, 64)
    print("Keystream (hex):", ks.hex())

    pt = b"HELLO_PRG_HMAC"
    ct = xor_bytes(pt, ks[:len(pt)])
    print("Cipher (hex):", ct.hex())
    print("Recovered:", xor_bytes(ct, ks[:len(pt)]))


## Napad na ponovitev keystreama
Dokaži, da če isti keystream uporabimo za dve različni sporočili P1 in P2, potem C1 XOR C2 = P1 XOR P2. Napiši program, ki to prikaže z obema PRG opcijama (LCG in HMAC) in pokaže, kako lahko z znanim fragmentom P1 odkriješ P2.

In [ ]:
# uporabi LCG ali HMAC PRG

ks = lcg_bytes(seed=123, a=1103515245, c=12345, m=2**31, n_bytes=32)
# ks = prg_hmac(b"seed", 32)

P1 = b"ATTACK AT DAWN!!!!"
P2 = b"MEET AT THE DOCK!!!"

C1 = xor_bytes(P1, ks[:len(P1)])
C2 = xor_bytes(P2, ks[:len(P2)])

# Če napadalec dobi C1 in C2:
xor_c = xor_bytes(C1, C2)
print("C1 XOR C2 (hex):", xor_c.hex())
# Če napadalec pozna del P1:
known_prefix = b"ATTACK"
recovered_part = xor_bytes(xor_c[:len(known_prefix)], known_prefix)
print("Recovered start of P2:", recovered_part)


## POENOSTAVLJEN “TLS-like” HANDSHAKE + SIMPLIFIED SECURE CHANNEL
Tu naredimo poenostavljen, izobraževalen protokol, ki prikaže glavne koncepte TLS: izmenjava ključa (Diffie-Hellman), derivacija session key, varna komunikacija s keystream + HMAC za integriteto. Brez SSL certifikatov — namesto tega prikažemo možnost overjanja s pre-shared key (PSK) ali s podpisom (lahko simuliraš)

#### Specifikacije (naloga)

1. Uporabi preddefiniran prost modul p (velik prim) in generator g.

2. Ključni koraki:

    - Client: izbere a (privatno), izračuna A = g^a mod p.

    - Server: izbere b, izračuna B = g^b mod p.

    - Zamenjata A in B.

    - Oba izračunata shared secret s = B^a mod p = A^b mod p.

3. Derivacija session key: K = SHA256(s_bytes || nonce_C || nonce_S).

4. Generiraj keystream ks = PRG_HMAC(K, n) in šifriraj s XOR.

5. Uporabi HMAC-SHA256(K, ciphertext || seq) za integriteto in preverjanje.

Napiši Python program, ki simulira klienta in strežnika v eni skripti, izvede handshake in pošlje sporočilo "Hello, server!" s HMAC over ciphertext in sequence number. Strežnik preveri HMAC in dešifrira.

#### Koda:

In [ ]:
# tls_like.py
import secrets, hashlib, hmac
from typing import Tuple

# 1) Poenostavljen DH parametri (majhen za demonstracijo; v resnici uporab 2048-bit)
p = int("F7E75FDC469067FFDC4E847C51F452DF", 16)  # 128-bit primer (le za učenje)
g = 2

def dh_keypair() -> Tuple[int, int]:
    priv = secrets.randbelow(p-2) + 1
    pub = pow(g, priv, p)
    return priv, pub

def derive_session_key(shared_secret_int: int, nonce_c: bytes, nonce_s: bytes) -> bytes:
    s_bytes = shared_secret_int.to_bytes((shared_secret_int.bit_length()+7)//8 or 1, 'big')
    return hashlib.sha256(s_bytes + nonce_c + nonce_s).digest()

def hmac_sha256(key: bytes, msg: bytes) -> bytes:
    return hmac.new(key, msg, hashlib.sha256).digest()

def xor_bytes(b1: bytes, b2: bytes) -> bytes:
    return bytes(x ^ y for x, y in zip(b1, b2))

if __name__ == "__main__":
    # Client
    a_priv, A = dh_keypair()
    nonce_c = secrets.token_bytes(8)

    # Server
    b_priv, B = dh_keypair()
    nonce_s = secrets.token_bytes(8)

    # Exchange (simulirano)
    # Both compute shared secret
    s_client = pow(B, a_priv, p)
    s_server = pow(A, b_priv, p)
    assert s_client == s_server

    K = derive_session_key(s_client, nonce_c, nonce_s)  # 32 bytes
    print("Derived session key (hex):", K.hex())

    # Encrypt message from client to server
    plaintext = b"Hello, server!"
    ks = prg_hmac(K, len(plaintext))
    ciphertext = xor_bytes(plaintext, ks)
    seq = (1).to_bytes(4, 'big')  # sequence number
    tag = hmac_sha256(K, ciphertext + seq)

    # Server side verify
    tag_v = hmac_sha256(K, ciphertext + seq)
    if not hmac.compare_digest(tag, tag_v):
        raise ValueError("HMAC mismatch! Message tampered.")
    recovered = xor_bytes(ciphertext, prg_hmac(K, len(plaintext)))
    print("Server recovered plaintext:", recovered)


: 

## 2FA
- Gesla se shranjujejo kot PBKDF2-HMAC-SHA256 (salt + 100k iteracij).

- TOTP implementacija uporablja HMAC-SHA1 (RFC 6238 uporablja SHA1/256/512; SHA1 je standard za kompatibilnost z Authenticator aplikacijami).

- Skrivnosti so shranjene v base32 obliki (kompatibilno z Google Authenticator).

- HOTP števec se povečuje na strežniku in ščiti pred replay zabeležitvijo porabljenih vrednosti.

- Za TOTP je privzeto okno +/- 1 časovni interval (30 s) — ta parameter lahko spreminjaš.

- Ta koda je izobraževalna; v produkciji uporabi preverjene knjižnice (npr. pyotp) in varno shranjevanje tajnih ključev.

In [ ]:
#!/usr/bin/env python3
"""
Two-factor authentication demo:
- registration (username + password + generate 2FA secret)
- login: password -> ask OTP (TOTP or HOTP)
- store persisted in JSON (simple)
Only uses Python standard library.
"""

import os
import json
import time
import hmac
import base64
import struct
import hashlib
import secrets
from typing import Optional

DB_FILE = "users_db.json"

# ---------------------------
# Utilities: password hashing
# ---------------------------
def hash_password(password: str, salt: Optional[bytes] = None, iterations: int = 100_000):
    if salt is None:
        salt = secrets.token_bytes(16)
    dk = hashlib.pbkdf2_hmac("sha256", password.encode(), salt, iterations)
    return {
        "salt": base64.b64encode(salt).decode(),
        "iterations": iterations,
        "digest": base64.b64encode(dk).decode()
    }

def verify_password(stored: dict, password: str) -> bool:
    salt = base64.b64decode(stored["salt"])
    iterations = stored["iterations"]
    expected = base64.b64decode(stored["digest"])
    dk = hashlib.pbkdf2_hmac("sha256", password.encode(), salt, iterations)
    return hmac.compare_digest(dk, expected)

# ---------------------------
# HOTP / TOTP implementation
# ---------------------------
def hotp_raw(key: bytes, counter: int, digits: int = 6) -> str:
    # key: raw bytes
    msg = struct.pack(">Q", counter)
    hmac_hash = hmac.new(key, msg, hashlib.sha1).digest()
    offset = hmac_hash[-1] & 0x0F
    code = ((hmac_hash[offset] & 0x7f) << 24) | ((hmac_hash[offset+1] & 0xff) << 16) | ((hmac_hash[offset+2] & 0xff) << 8) | (hmac_hash[offset+3] & 0xff)
    otp = code % (10 ** digits)
    return str(otp).zfill(digits)

def totp_raw(key: bytes, for_time: Optional[int] = None, step: int = 30, digits: int = 6) -> str:
    if for_time is None:
        for_time = int(time.time())
    counter = for_time // step
    return hotp_raw(key, counter, digits)

# ---------------------------
# User store (JSON-backed)
# ---------------------------
def load_db():
    if not os.path.exists(DB_FILE):
        return {}
    with open(DB_FILE, "r") as f:
        return json.load(f)

def save_db(db):
    with open(DB_FILE, "w") as f:
        json.dump(db, f, indent=2)

# ---------------------------
# Helpers: base32 secret
# ---------------------------
def generate_base32_secret(length_bytes: int = 20) -> str:
    raw = secrets.token_bytes(length_bytes)
    return base64.b32encode(raw).decode('utf-8').strip('=')

def base32_to_bytes(b32: str) -> bytes:
    # add padding back
    padding = '=' * ((8 - len(b32) % 8) % 8)
    return base64.b32decode(b32 + padding)

# ---------------------------
# High-level user operations
# ---------------------------
def register_user(db, username: str, password: str, mode: str = "TOTP"):
    if username in db:
        raise ValueError("User exists")
    pwd_record = hash_password(password)
    secret = generate_base32_secret(20)  # 160-bit secret
    user = {
        "password": pwd_record,
        "2fa": {
            "mode": mode,           # "TOTP" or "HOTP"
            "secret": secret,
            "hotp_counter": 0      # only for HOTP
        }
    }
    db[username] = user
    save_db(db)
    return user

def get_provisioning_uri(username: str, secret_b32: str, issuer: str = "DemoApp", algo: str = "SHA1", digits: int = 6, period: int = 30):
    # Standard otpauth URI (works with Authenticator apps):
    # otpauth://totp/ISSUER:USERNAME?secret=BASE32&issuer=ISSUER&algorithm=SHA1&digits=6&period=30
    label = f"{issuer}:{username}"
    uri = f"otpauth://totp/{label}?secret={secret_b32}&issuer={issuer}&algorithm={algo}&digits={digits}&period={period}"
    return uri

def verify_totp_for_user(db, username: str, code: str, window: int = 1, step: int = 30) -> bool:
    # window: number of steps to check backwards/forwards
    if username not in db:
        return False
    rec = db[username]["2fa"]
    if rec["mode"] != "TOTP":
        return False
    key = base32_to_bytes(rec["secret"])
    now = int(time.time())
    # check -window .. +window
    for w in range(-window, window+1):
        t = now + w * step
        if hmac.compare_digest(totp_raw(key, t, step), code):
            return True
    return False

def verify_hotp_for_user(db, username: str, code: str, look_ahead: int = 5) -> bool:
    # look_ahead: allow server to search upcoming counters to allow slight desync
    if username not in db:
        return False
    rec = db[username]["2fa"]
    if rec["mode"] != "HOTP":
        return False
    key = base32_to_bytes(rec["secret"])
    ctr = rec.get("hotp_counter", 0)
    for i in range(look_ahead+1):
        candidate = hotp_raw(key, ctr + i)
        if hmac.compare_digest(candidate, code):
            # accept and advance counter to counter after used value
            rec["hotp_counter"] = ctr + i + 1
            save_db(db)
            return True
    return False

# ---------------------------
# Simple CLI flow
# ---------------------------
def demo():
    print("=== Simple 2FA demo ===")
    db = load_db()

    while True:
        print("\nOptions: [1] register  [2] login  [3] show users  [4] exit")
        choice = input("Choice: ").strip()
        if choice == "1":
            uname = input("username: ").strip()
            pwd = input("password: ").strip()
            mode = input("2FA mode (TOTP/HOTP) [TOTP]: ").strip().upper() or "TOTP"
            try:
                user = register_user(db, uname, pwd, mode)
            except ValueError as e:
                print("Error:", e)
                continue
            print("User registered.")
            print("2FA secret (base32):", user["2fa"]["secret"])
            print("Provisioning URI (paste into Authenticator app):")
            print(get_provisioning_uri(uname, user["2fa"]["secret"]))
            continue

        if choice == "2":
            uname = input("username: ").strip()
            pwd = input("password: ").strip()
            db = load_db()  # reload to pick up updates
            if uname not in db:
                print("Unknown user")
                continue
            if not verify_password(db[uname]["password"], pwd):
                print("Password incorrect")
                continue
            print("Password OK. Now enter 2FA code.")
            mode = db[uname]["2fa"]["mode"]
            code = input("2FA code: ").strip()
            if mode == "TOTP":
                ok = verify_totp_for_user(db, uname, code)
            else:
                ok = verify_hotp_for_user(db, uname, code)
            if ok:
                print("2FA OK — login successful.")
            else:
                print("2FA failed.")
            continue

        if choice == "3":
            db = load_db()
            print("Users in DB:", list(db.keys()))
            continue

        if choice == "4":
            print("bye")
            break

        print("Unknown option.")

if __name__ == "__main__":
    demo()


## Izračun trenutnega TOTP (enkratno geslo)
Izračunamo TOTP po RFC6238: uporabimo base32 skrivnost S, časovni korak 30 s, HMAC-SHA1 in dinamično truncation. Rezultat je 6-mestna številka.

In [ ]:
# totp_example.py
import time, hmac, hashlib, struct, base64

def base32_to_bytes(b32: str) -> bytes:
    padding = '=' * ((8 - len(b32) % 8) % 8)
    return base64.b32decode(b32 + padding)

def hotp_from_key_and_counter(key: bytes, counter: int, digits: int = 6) -> str:
    msg = struct.pack(">Q", counter)
    hmac_hash = hmac.new(key, msg, hashlib.sha1).digest()
    offset = hmac_hash[-1] & 0x0F
    code = ((hmac_hash[offset] & 0x7f) << 24) | ((hmac_hash[offset+1] & 0xff) << 16) | ((hmac_hash[offset+2] & 0xff) << 8) | (hmac_hash[offset+3] & 0xff)
    otp = code % (10 ** digits)
    return str(otp).zfill(digits)

def totp(secret_b32: str, step: int = 30, digits: int = 6, for_time: int = None) -> str:
    if for_time is None:
        for_time = int(time.time())
    counter = for_time // step
    key = base32_to_bytes(secret_b32)
    return hotp_from_key_and_counter(key, counter, digits)

if __name__ == "__main__":
    # primer skrivnosti (base32). Na izpitu boste dobili S.
    S = "JBSWY3DPEHPK3PXP"  # primer (base32)
    print("Current TOTP:", totp(S))
    # Če želiš reproducirati za točen čas:
    # print("TOTP at t=1600000000:", totp(S, for_time=1600000000))


## Preveri integriteto datoteke in kaj se zgodi, če napadalec spremeni en bit
Metoda: izračunamo npr. SHA-256 hash vsebine in primerjamo s priloženim (reference) hashom. Če se spremeni en bit, se hash drastično spremeni (avalanche effect) → integriteta ne drži.

In [ ]:
# integrity_check.py
import hashlib

def sha256_hex(data: bytes) -> str:
    return hashlib.sha256(data).hexdigest()

def check_integrity(content: bytes, expected_hash_hex: str) -> bool:
    h = sha256_hex(content)
    print("Computed hash:", h)
    print("Expected hash:", expected_hash_hex)
    return h.lower() == expected_hash_hex.lower()

if __name__ == "__main__":
    original = b"Hello, this is the original file content.\n"
    expected = sha256_hex(original)
    print("=== Original check ===")
    ok = check_integrity(original, expected)
    print("Integrity OK?", ok)

    # simulate one-bit flip in the content
    tampered = bytearray(original)
    tampered[0] ^= 0x01  # flip least significant bit of first byte
    print("\n=== After tampering (one bit flipped) ===")
    ok2 = check_integrity(bytes(tampered), expected)
    print("Integrity OK after tampering?", ok2)


## Preprosta MITM (man-in-the-middle) simulacija
Dve verziji:

- A) Brez HMAC: MITM lahko spremeni sporočilo neopazno.

- B) Z HMAC (deljena skrivnost K): MITM ne more pravilno ponastaviti HMAC, zato bo sprememba odkrita (razen če attacker pozna K).

Koda prikazuje oboje.

In [ ]:
# mitm_simulation.py
import hmac, hashlib

def hmac_sha256(key: bytes, data: bytes) -> bytes:
    return hmac.new(key, data, hashlib.sha256).digest()

# Scenario A: brez integritete (MITM uspešno spremeni)
def scenario_no_hmac():
    print("Scenario A: NO HMAC (vulnerable)")
    client_msg = b"Transfer $1000 to account X"
    # MITM intercepts and modifies:
    mitm_msg = b"Transfer $9000 to account X"
    # Server receives modified message - no way to detect
    print("Client sent :", client_msg)
    print("MITM forwarded (modified):", mitm_msg)
    print("Server accepts modified message (no HMAC)\n")

# Scenario B: z HMAC: server preveri tag
def scenario_with_hmac():
    print("Scenario B: WITH HMAC (protected if key secret)")
    K = b"supersecretkey"  # shared key between client and server
    client_msg = b"Transfer $1000 to account X"
    tag = hmac_sha256(K, client_msg)
    # MITM intercepts and modifies message, but cannot recompute tag
    mitm_msg = b"Transfer $9000 to account X"
    mitm_tag = tag  # attacker forwards original tag (doesn't match)
    # Server verifies
    expected_tag = hmac_sha256(K, mitm_msg)
    print("Client sent msg :", client_msg)
    print("Client tag (hex):", tag.hex())
    print("MITM forwarded msg (modified):", mitm_msg)
    print("MITM forwarded tag (unchanged):", mitm_tag.hex())
    if hmac.compare_digest(mitm_tag, expected_tag):
        print("Server: HMAC OK (message accepted)") 
    else:
        print("Server: HMAC mismatch — modification detected (message rejected)")

if __name__ == "__main__":
    scenario_no_hmac()
    scenario_with_hmac()


## XOR z keystreamom (šifriranje/dešifriranje)
Tokovne šifre: cipher = plaintext XOR keystream. Dešifriranje je enako: plaintext = cipher XOR keystream. Tu sta dva primera PRG za keystream: poenostavljen LCG (nevaren) in HMAC-based PRG (varnejši).

In [ ]:
# xor_keystream.py
import hmac, hashlib, secrets

def xor_bytes(a: bytes, b: bytes) -> bytes:
    return bytes(x ^ y for x,y in zip(a,b))

# 1) LCG-based keystream (nejaka primerna dolžina)
def lcg_keystream(seed: int, a: int, c: int, m: int, n: int) -> bytes:
    state = seed % m
    out = bytearray()
    for _ in range(n):
        state = (a * state + c) % m
        out.append(state % 256)
    return bytes(out)

# 2) HMAC-based PRG (HKDF-like expand)
def prg_hmac(seed: bytes, n_bytes: int) -> bytes:
    out = bytearray()
    counter = 1
    last = b""
    while len(out) < n_bytes:
        mac = hmac.new(seed, last + bytes([counter]), hashlib.sha256).digest()
        out.extend(mac)
        last = mac
        counter += 1
    return bytes(out[:n_bytes])

if __name__ == "__main__":
    plaintext = b"This is secret!"
    n = len(plaintext)

    # Example A: LCG keystream (educational only)
    ks1 = lcg_keystream(seed=42, a=1103515245, c=12345, m=2**31, n=n)
    cipher1 = xor_bytes(plaintext, ks1)
    recovered1 = xor_bytes(cipher1, ks1)
    print("LCG Cipher (hex):", cipher1.hex())
    print("Recovered:", recovered1)

    # Example B: HMAC-based PRG (better)
    seed = b"shared secret seed"
    ks2 = prg_hmac(seed, n)
    cipher2 = xor_bytes(plaintext, ks2)
    recovered2 = xor_bytes(cipher2, ks2)
    print("HMAC-PRG Cipher (hex):", cipher2.hex())
    print("Recovered:", recovered2)
